# Ambientaciones de Lotes Bajos 

In [1]:
import ee
ee.Initialize()
import ambientacionesff as ambff
import config
#Genero la carpeta temporal
import os 
from os import path

dirName = 'temp'

if path.exists("temp") == False:
 os.mkdir(dirName)


# Mapa para visualizaciones

In [2]:
import geemap
#import geemap.eefolium as geemap
Map = geemap.Map(center=[-36,-63], zoom=4)
url = 'http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Map', attribution='Google')
Map.addLayerControl()


Map

Map(center=[-36, -63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
import ipywidgets as widgets

uploader = widgets.FileUpload(button_style='primary',  accept='.kml',description="KML")
display(uploader)

FileUpload(value={}, accept='.kml', button_style='primary', description='KML')

In [5]:
import config
import ee
import json
import tempfile
import geopandas as gpd
from ipywidgets import Checkbox, HBox, VBox
import ipywidgets as widgets

import importlib

style = {'description_width': 'initial'}

output = widgets.Output()
output1 = widgets.Output()


subirkml = widgets.Button(
    description='Agregar Lote',
    button_style='primary',
    tooltip='Click para buscar imagenes',
    style=style
)
   

def subirkml_clicked(b):
    
 with output:
        print("Iniciando...")
 llaves = uploader.value.keys()
 
 if llaves.__len__() != 0:
    try:
        nombre = list(llaves)
        nombre_archivo = nombre[0]

        contents = uploader.value[nombre_archivo]['content']
        contents.decode("utf-8") 

        f = open("temp/lote.kml", "w")
        f.write(contents.decode("utf-8"))
        f.close()

        geojson,df1,nombrelote = ambff.ingresar_poligono(nombre_archivo,"temp/lote.kml")
        with output:
            print("Archivo cargado")
        #Grafico
        d = json.loads(geojson)
        geometria = (d['features'][0]["geometry"]["coordinates"])
        lote = ee.Geometry.Polygon(geometria[0])
        config.lote = lote
        Map.addLayer(lote,{}, "Lotes")
        Map.centerObject(lote, 14)
        
        datos_lote,coleccion,lote = ambff.busqueda_imagenes(output,config.lote,2017,2020,1,12)
        ambff.graficar_series(output1,datos_lote)
    
    except:
     with output:
         print("No se cargo un archivo valido, por favor revise el geojson o visite https://github.com/francofrolla/ ")
    
       
    
 if llaves.__len__() == 0:
  lote = "error"
  with output:
   print("No se selecciono ningun archivo")

 return 
 
  

subirkml.on_click(subirkml_clicked)
graficas = HBox([subirkml,output])
VBox([graficas,output1])

C:\Users\Usuario\miniconda3\envs\AmbientadorFFNP\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Archivo cargado


# Genero composición mensual de Sentinel 2, descargo datos y grafico

In [ ]:
datos_lote,coleccion,lote = ambff.busqueda_imagenes(output,config.lote,2017,2020,1,12)
ambff.graficar_series(output1,datos_lote)


In [ ]:
from ipywidgets import Checkbox, HBox, VBox
from IPython.display import display, clear_output
import ipyleaflet
import ipywidgets as widgets


#CONSTANTES DEL MAPA
#palette = ['#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718', '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201', '#004C00', '#023B01', '#012E01', '#011D01', '#011301']
#Map.add_colorbar_branca(colors=palette,vmin=0.2,vmax=0.7)
output_widget = widgets.Output(layout={'border': '1px solid black',"font_weight": "5px"})
output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomleft')
Map.add_control(output_control)

#selfechas = widgets.Dropdown(
#    options=config.listaid,
#    value=config.listaid[0],
#    description='Seleccione Fecha:',
#    disabled=False,
#)
years = [2017,2018,2019,2020]
mes = [1,2,3,4,5,6,7,8,9,10,11,12]

selyear = widgets.Dropdown(
    options=years,
    value=years[0],
    description='Año:',
    disabled=False,
)

selmes = widgets.Dropdown(
    options=mes,
    value=mes[0],
    description='Mes:',
    disabled=False,
)


boton1 = widgets.Button(description='Graficar NDVI',button_style='primary')
boton2 = widgets.Button(description='Graficar',button_style='primary')

out1 = widgets.Output()


def graficar_onclicked(b):
 with out1:
  texto = "Cargando..."
 
 coleccion1 = coleccion.filterMetadata('year', 'equals', str(selyear.value))
 coleccion1 = coleccion1.filterMetadata('month', 'equals', selmes.value)
 escena = ee.Image(coleccion1.first().select(["NDVI"]))

 params = escena.reduceRegion(
           reducer= ee.Reducer.percentile([10, 90]), 
           geometry= lote, 
           scale= 10,
           )

 parametros = params.getInfo()
 min_valor = [parametros['NDVI_p10']]
 max_valor = [parametros['NDVI_p90']]

 vis_params = {
              'min': min_valor,
              'max': max_valor,
              'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']}
 
 fecha = str(selyear.value)+" "+str(selmes.value)
 with out1:
  texto = "Graficando imagen "+fecha+" NDVI min:"+ str(min_valor) +" NDVI max:"+ str(max_valor) 
  print(texto)

 Map.addLayer(escena, vis_params, fecha)

 with output_widget:
  clear_output()
  print(texto)

def graficar_cv_onclicked(b):
 #Borro lo anterior
 coleccion1 = coleccion.filterMetadata('year', 'equals', str(selyear.value))
 coleccion1 = coleccion1.filterMetadata('month', 'equals', selmes.value)
 escena = ee.Image(coleccion1.first())
 
 
 fecha = str(selyear.value)+" "+str(selmes.value)

 params = escena.select(["B4","B3","B2"]).reduceRegion(
      reducer= ee.Reducer.percentile([5, 95]), 
      geometry= lote, 
      scale= 10,
      )
 parametros = params.getInfo()
 min_escena = [parametros['B4_p5'], parametros['B3_p5'], parametros['B2_p5']]
 max_escena = [parametros['B4_p95'], parametros['B3_p95'], parametros['B2_p95']]

 vis_params = {
              'bands':['B4', 'B3', 'B2'],
              'min': min_escena,
              'max': max_escena,
               }

 with out1:
  texto = "Graficando imagen "+fecha 
  print(fecha)

 Map.addLayer(escena, vis_params, fecha)

 with output_widget:
  clear_output()
  print("Graficando" + texto)
  

boton1.on_click(graficar_onclicked)
boton2.on_click(graficar_cv_onclicked)

botones = HBox([selyear,selmes,boton1,boton2])
VBox([botones,out1])

In [ ]:
%%time

#Descargar imagenes via link

años=[2018]
meses=[10]

def funcion_sentinel(coleccion,años,meses):
    laimagen = ee.Image()
    for i in meses:
     coleccion1 = coleccion.filterMetadata('year', 'equals', años)
     coleccion1 = coleccion1.filterMetadata('month', 'equals', i)
     imagen = ee.Image(coleccion1.first().select(["NDVI"])).rename(str(i))
     laimagen = laimagen.addBands(imagen)
    
    nombre = str(años)
    print(nombre)
    print("Descargando...")
    path = laimagen.getDownloadUrl({
      'name': nombre,
      'scale' : 10,
      'region' : lote})
    print(path)
    return laimagen, nombre



#Año tiene que estar en formato string
#Meses en int

Recorte, nombre = funcion_sentinel(coleccion,str(años[0]),meses)

# Aca se termino el script